In [ ]:
import numpy as np
import pandas as pd
import os
import csv
import ast
import seaborn as sns
from matplotlib import pyplot as plt
import wquantiles 

from tqdm import tqdm
from wquantiles import quantile
from openfisca_survey_manager.utils import asof

from openfisca_france_indirect_taxation import FranceIndirectTaxationTaxBenefitSystem
from openfisca_france_indirect_taxation.surveys import SurveyScenario
from openfisca_france_indirect_taxation.examples.utils_example import (
    dataframe_by_group,
    df_weighted_average_grouped,
    wavg)
from openfisca_france_indirect_taxation.build_survey_data.utils import weighted_sum
from openfisca_france_indirect_taxation.Calage_consommation_bdf import get_inflators_by_year, get_cn_aggregates
from openfisca_france_indirect_taxation.projects.TVA.Utils import weighted_quantiles
from openfisca_france_indirect_taxation.utils import assets_directory, get_input_data_frame

In [ ]:
simulated_variables = ['aise',
'depenses_totales',
'depenses_tot',
'rev_disponible',
 'niveau_de_vie',
 'niveau_vie_decile',
 'decile_indiv_niveau_vie',
 'ocde10',
 'pondmen',
 'nactifs',
 'npers',
 'identifiant_menage']

In [ ]:
# Première simulation : données BdF 2017 brutes
year = 2017
data_year = 2017
tax_benefit_system = FranceIndirectTaxationTaxBenefitSystem()

survey_scenario = SurveyScenario.create(
    tax_benefit_system = tax_benefit_system,
    year = year,
    data_year = data_year
    )

In [ ]:
def epargne(df):
    df['part_epargnants'] = (df['rev_disponible'] >= df['depenses_tot']).astype(float)
    df['aise_1_2_3'] = (df['aise'] <= 3).astype(float)
    df_by_decile = df_weighted_average_grouped(dataframe = df, groupe = 'decile_indiv_niveau_vie', varlist = ['depenses_tot','rev_disponible','niveau_de_vie','ocde10','part_epargnants','aise_1_2_3'])
    df_by_decile['taux_epargne'] = 1 - (df_by_decile ['depenses_tot'] / df_by_decile ['rev_disponible'])

    return(df_by_decile)
    

In [ ]:
bdf_brut_2017 = survey_scenario.create_data_frame_by_entity(simulated_variables, period = 2017)['menage']
bdf_brut_2017_by_decile = epargne(bdf_brut_2017)

In [ ]:
bdf_brut_2017_by_decile

In [ ]:
# Deuxième simulation : données BdF calées sur la compta nat en 2017 
inflators_by_year = get_inflators_by_year(rebuild = True, year_range = range(2017, 2025), data_year = data_year)
inflation_kwargs = dict(inflator_by_variable = inflators_by_year[year])

survey_scenario = SurveyScenario.create(
    inflation_kwargs =  inflation_kwargs,
    tax_benefit_system = tax_benefit_system,
    year = year,
    data_year = data_year
    )

In [ ]:
bdf_cale_2017 = survey_scenario.create_data_frame_by_entity(simulated_variables, period = 2017)['menage']
bdf_cale_2017_by_decile = epargne(bdf_cale_2017)

In [ ]:
bdf_cale_2017_by_decile

In [ ]:
# Comparaison avec l'ERFS 
erfs_path = "C:/Users/veve1/OneDrive/Documents/ENSAE 3A/Memoire MiE/Data/erfs_fpr/2017/csv"
erfs_menage_2017 = pd.read_csv(os.path.join(erfs_path,"fpr_menage_2017.csv"), sep = ";")
erfs_mrf17 = pd.read_csv(os.path.join(erfs_path,"fpr_mrf17e17t4.csv"), sep = ";") 

erfs_2017 = erfs_menage_2017.merge(erfs_mrf17, how = "left", left_on='ident17', right_on = 'ident17')
erfs_2017.columns = erfs_2017.columns.str.lower()
erfs_2017.rename({'wprm' : 'pondmen'}, axis = 1, inplace= True)
erfs_2017['pondindiv'] = erfs_2017['nbind'] * erfs_2017['pondmen']

erfs_2017['niveau_de_vie'] = erfs_2017['revdispm'] / erfs_2017['nb_uci']
erfs_2017['decile_indiv_niveau_vie'] = weighted_quantiles(erfs_2017['niveau_de_vie'], labels = np.arange(1,11), weights = erfs_2017['pondindiv'], return_quantiles=False)
erfs_2017_by_decile = df_weighted_average_grouped(erfs_2017, groupe = 'decile_indiv_niveau_vie', varlist = ['revdispm','niveau_de_vie','nb_uci'])

In [ ]:
(erfs_2017['revdispm'] * erfs_2017['pondmen']).sum() * 1E-9

In [ ]:
cn_aggregates = get_cn_aggregates(2017)
cn_aggregates.loc[cn_aggregates.index == 'rev_disponible','conso_CN_2017'] * 1e-9

In [ ]:
bdf_brut_2017_by_decile

In [ ]:
erfs_2017_by_decile

In [ ]:
bdf_cale_2017_by_decile

In [ ]:
bdf_brut_2017_by_decile.rename({'niveau_de_vie' : 'niveau_de_vie_bdf' }, axis = 1, inplace = True)
erfs_2017_by_decile.rename({'niveau_de_vie' : 'niveau_de_vie_erfs'}, axis= 1, inplace = True)

In [ ]:
base_calage = bdf_brut_2017_by_decile.merge(erfs_2017_by_decile, how = 'left', left_index = True, right_index= True)
base_calage['ratio_erfs_bdf'] = base_calage['niveau_de_vie_erfs'] / base_calage['niveau_de_vie_bdf']
base_calage

In [ ]:
input_bdf_2017 = get_input_data_frame(2017)
input_bdf_2017_grouped = input_bdf_2017.filter(like='poste_').groupby(lambda col: '_'.join(col.split('_')[:3]), axis=1).sum()

to_drop = [col for col in input_bdf_2017_grouped.columns.tolist() if col.startswith('poste_17') or col.startswith('poste_18')]
input_bdf_2017_grouped.drop(columns= to_drop, axis = 1, inplace= True)
input_bdf_2017_grouped = pd.concat([input_bdf_2017[['pondmen','npers','ident_men','rev_disponible','ocde10','aise','stalog','typmen']], input_bdf_2017_grouped], axis=1)

## Méthode 1 : redressement des revenus sur la base d’un filtre de cohérence revenu/consommation

Tiré de Bellamy et al. (2009): "Le redressement du revenu basé sur le filtre de cohérence revenu - consommation est le
suivant :
    (i) sur l’ensemble du fichier de l’enquête les très faibles revenus sont redressés : le
niveau de vie d’un ménage déclarant moins de 300 € par mois et par UC est
ramené automatiquement à 300 € par mois et par UC. Ce cas concerne 16
ménages sur les 10 240 de l’échantillon.
    (ii) On définit ensuite une "consommation courante" en éliminant les
consommations exceptionnelles.
    (iii) Si cette consommation courante est supérieure de plus de 20 % au revenu
courant (revenu constaté une fois ce dernier éventuellement mis à 300 € par
mois et par UC en (i)), soit pour 15,3 % des ménages, on effectue le test
suivant, analogue à celui utilisé par Loisy (1999).

Si le ménage :
1 - déclare être « à l’aise » financièrement, que « ça va » ou que « c’est juste, mais il
faut faire attention »
et
2 - déclare mettre de l’argent de côté ou que revenus et dépenses s’équilibrent
alors on remonte le revenu calculé en (i) au niveau de la consommation courante (ii). Dans
les autres cas, on laisse le revenu (i) et la consommation inchangés. Au total 10,7 % des
ménages sont affectés par ce redressement. On notera que ce traitement touche a priori
tous les ménages, quel que soit leur niveau de vie initial."

In [ ]:
input_bdf_2017_grouped.drop('poste_04_2', axis= 1, inplace= True)        # on retire les loyers imputés des dépenses
input_bdf_2017_grouped = input_bdf_2017_grouped.apply(pd.to_numeric, errors='coerce')

liste_poste = [col for col in input_bdf_2017_grouped.columns.tolist() if col.startswith('poste')]
for poste in liste_poste:
   results = weighted_quantiles(input_bdf_2017_grouped['{}'.format(poste)], labels = np.arange(1,21), weights = input_bdf_2017_grouped['pondmen'], return_quantiles=True)
   input_bdf_2017_grouped['conso_courante_{}'.format(poste)] = input_bdf_2017_grouped['{}'.format(poste)].clip(upper=results[1][19])

input_bdf_2017_grouped['depenses_tot'] = input_bdf_2017_grouped[liste_poste].sum(axis = 1) 
input_bdf_2017_grouped['conso_courante_totale'] = input_bdf_2017_grouped.filter(like = 'conso_courante').sum(axis = 1)

input_bdf_2017_grouped['rev_disponible_2'] = input_bdf_2017_grouped['rev_disponible'].clip(lower= 3600 * input_bdf_2017_grouped['ocde10'])   

In [ ]:
len(input_bdf_2017_grouped.loc[input_bdf_2017_grouped['rev_disponible_2'] > input_bdf_2017_grouped['rev_disponible']])

Il y a 137 ménages dont le niveau de vie est inférieur à 300€ par mois par UC.

In [ ]:
input_bdf_2017_grouped['TEST'] = input_bdf_2017_grouped['conso_courante_totale'] > 1.2 * input_bdf_2017_grouped['rev_disponible_2']
input_bdf_2017_grouped['rev_disponible_3'] = input_bdf_2017_grouped['TEST']*(input_bdf_2017_grouped['aise'] <= 3)*input_bdf_2017_grouped['conso_courante_totale'] + input_bdf_2017_grouped['TEST'] * (input_bdf_2017_grouped['aise']>3) * input_bdf_2017_grouped['rev_disponible_2'] + (1 - input_bdf_2017_grouped['TEST']) * input_bdf_2017_grouped['rev_disponible_2']

In [ ]:
input_bdf_2017_grouped['TEST'].sum(axis = 0)

Il y a 1232 (825 quand le seuil est à p90) ménages dont la consommation courante est supérieure de plus de 20% à leur revenu disponible

In [ ]:
(input_bdf_2017_grouped['TEST']*(input_bdf_2017_grouped['aise'] <= 3)).sum()

Dont 890 (559 quand le seuil est à p90) qui déclarent être « à l’aise » financièrement, que « ça va » ou que « c’est juste, mais il
faut faire attention ».

In [ ]:
input_bdf_2017_grouped['part_epargnants'] = input_bdf_2017_grouped['rev_disponible_3'] >= input_bdf_2017_grouped['depenses_tot'] 
input_bdf_2017_grouped['part_epargnants_initiale'] = input_bdf_2017_grouped['rev_disponible'] >= input_bdf_2017_grouped['depenses_tot']

In [ ]:
input_bdf_2017_grouped['pondindiv'] = input_bdf_2017_grouped['npers'] * input_bdf_2017_grouped['pondmen']
for rev in ['','_2','_3']:
    input_bdf_2017_grouped['niveau_de_vie{}'.format(rev)] = input_bdf_2017_grouped['rev_disponible{}'.format(rev)] / input_bdf_2017_grouped['ocde10']
    
input_bdf_2017_grouped['decile_indiv_niveau_vie_3'] = weighted_quantiles(input_bdf_2017_grouped['niveau_de_vie_3'], labels = np.arange(1,11), weights = input_bdf_2017_grouped['pondindiv'], return_quantiles=False)
input_bdf_2017_by_decile = df_weighted_average_grouped(input_bdf_2017_grouped, groupe = 'decile_indiv_niveau_vie_3', varlist = ['niveau_de_vie_3','rev_disponible_3','depenses_tot','pondmen','part_epargnants','part_epargnants_initiale'])

input_bdf_2017_by_decile['taux_epargne'] = 1 - input_bdf_2017_by_decile['depenses_tot'] / input_bdf_2017_by_decile['rev_disponible_3'] 

In [ ]:
input_bdf_2017_by_decile

In [ ]:
erfs_2017_by_decile

## Méthode 2 : Imputation des données de consommation par strates

### Méthode 2.1 : En utilisant le revenu disponible de BdF (brut)

In [ ]:
to_drop = [col for col in input_bdf_2017.columns.tolist() if col.startswith('poste_17') or col.startswith('poste_18')]
input_bdf_2017.drop(to_drop, axis = 1, inplace = True)
input_bdf_2017['niveau_de_vie'] = input_bdf_2017['rev_disponible'] / input_bdf_2017['ocde10']
input_bdf_2017['niveau_de_vie'] = input_bdf_2017['niveau_de_vie'].astype(float)
input_bdf_2017['pondmen'] = input_bdf_2017['pondmen'].astype(float)
input_bdf_2017['npers'] = input_bdf_2017['npers'].astype(float)

input_bdf_2017['pondindiv'] = input_bdf_2017['npers'] * input_bdf_2017['pondmen'] 

input_bdf_2017['decile_indiv_niveau_vie'] = weighted_quantiles(data = input_bdf_2017['niveau_de_vie'], labels = np.arange(1,11), weights = input_bdf_2017['pondindiv'], return_quantiles=False)
input_bdf_2017['stalog_2'] = input_bdf_2017['stalog'].replace({1:1, 3:2, 4:2, 5:2})

In [ ]:
def assign_strate_ines(row):
    if row['typmen'] == 5:
        return 'T5'
    elif row['typmen'] == 2:
        return f'T2_D{int(row["decile_indiv_niveau_vie"])}'
    elif row['typmen'] in [1, 3, 4]:
        return f'T{int(row["typmen"])}_D{int(row["decile_indiv_niveau_vie"])}_S{int(row["stalog_2"])}'
    else:
        return 'Other'

input_bdf_2017['strate_ines'] = input_bdf_2017.apply(assign_strate_ines, axis=1)

In [ ]:
input_bdf_2017['strate_ines'].nunique()

In [ ]:
input_bdf_2017[['decile_indiv_niveau_vie','typmen','stalog','strate_ines']]

In [ ]:
input_bdf_2017['strate_ines'].value_counts()

In [ ]:
liste_var = ['rev_disponible'] + [col for col in input_bdf_2017.columns.tolist() if col.startswith('poste_')]

input_bdf_2017_by_strate = df_weighted_average_grouped(input_bdf_2017, groupe = 'strate_ines', varlist = liste_var)
for poste in [col for col in input_bdf_2017.columns.tolist() if col.startswith('poste_')] : 
    input_bdf_2017_by_strate['part_{}'.format(poste)] = input_bdf_2017_by_strate['{}'.format(poste)] / input_bdf_2017_by_strate['rev_disponible']
    
part_conso_by_strat = input_bdf_2017_by_strate.filter(like = 'part').reset_index()

In [ ]:
new_input_bdf_2017 = input_bdf_2017.merge(part_conso_by_strat, how = 'left', left_on = 'strate_ines', right_on = 'strate_ines')
for poste in [col for col in input_bdf_2017.columns.tolist() if col.startswith('poste_')] : 
    new_input_bdf_2017['new_{}'.format(poste)] = new_input_bdf_2017['part_{}'.format(poste)] * new_input_bdf_2017['rev_disponible']
    
liste_new_poste = [col for col in new_input_bdf_2017.columns.tolist() if col.startswith('new_poste')]
new_input_bdf_2017['depenses_tot'] = new_input_bdf_2017[liste_new_poste].sum(axis = 1)

new_input_bdf_2017_by_decile = df_weighted_average_grouped(dataframe = new_input_bdf_2017, groupe = 'decile_indiv_niveau_vie', varlist =['depenses_tot','rev_disponible'])
new_input_bdf_2017_by_decile['taux_epargne'] = 1 - new_input_bdf_2017_by_decile['depenses_tot'] / new_input_bdf_2017_by_decile['rev_disponible']

In [ ]:
new_input_bdf_2017_by_decile

In [ ]:
def imputation_depenses_ines(input_df):
    to_drop = [col for col in input_df.columns.tolist() if col.startswith('poste_17') or col.startswith('poste_18')]
    input_df.drop(to_drop, axis = 1, inplace = True)
    
    input_df['niveau_de_vie'] = input_df['rev_disponible'] / input_df['ocde10']
    input_df['niveau_de_vie'] = input_df['niveau_de_vie'].astype(float)
    input_df['pondmen'] = input_df['pondmen'].astype(float)
    input_df['decile_indiv_niveau_vie'] = weighted_quantiles(data = input_df['niveau_de_vie'], labels = np.arange(1,11), weights = input_df['pondmen'], return_quantiles=False)
    input_df['stalog_2'] = input_df['stalog'].replace({1:1, 3:2, 4:2, 5:2})
    
    input_df['strate_ines'] = input_df.apply(assign_strate_ines, axis=1)
    liste_var = ['rev_disponible'] + [col for col in input_df.columns.tolist() if col.startswith('poste_')]

    input_df_by_strate = df_weighted_average_grouped(input_df, groupe = 'strate_ines', varlist = liste_var)
    for poste in [col for col in input_df.columns.tolist() if col.startswith('poste_')] : 
        input_df_by_strate['part_{}'.format(poste)] = input_df_by_strate['{}'.format(poste)] / input_df_by_strate['rev_disponible']
    part_conso_by_strat = input_df_by_strate.filter(like = 'part').reset_index()
    
    new_input_df = input_df.merge(part_conso_by_strat, how = 'left', left_on = 'strate_ines', right_on = 'strate_ines')
    for poste in [col for col in input_df.columns.tolist() if col.startswith('poste_')] : 
        new_input_df['new_{}'.format(poste)] = new_input_df['part_{}'.format(poste)] * new_input_df['rev_disponible']
        
    liste_new_poste = [col for col in new_input_df.columns.tolist() if col.startswith('new_poste')]
    new_input_df['depenses_tot'] = new_input_df[liste_new_poste].sum(axis = 1)

    new_input_df_by_decile = df_weighted_average_grouped(dataframe = new_input_df, groupe = 'decile_indiv_niveau_vie', varlist =['depenses_tot','rev_disponible'])
    new_input_df_by_decile['taux_epargne'] = 1 - new_input_df_by_decile['depenses_tot'] / new_input_df_by_decile['rev_disponible']
    
    return(new_input_df,new_input_df_by_decile)

### Méthode 2.2 : En utilisant les consos et revenus BdF calés en 2017

In [ ]:
input_bdf_cale_2017 = survey_scenario.create_data_frame_by_entity(liste_var + ['ocde10','pondmen','stalog','typmen'], period = 2017)['menage']

In [ ]:
new_input_bdf_cale_2017, new_input_bdf_cale_2017_by_decile = imputation_depenses_ines(input_bdf_cale_2017)

In [ ]:
new_input_bdf_cale_2017_by_decile

In [ ]:
new_input_bdf_cale_2017_by_decile[['depenses_tot','rev_disponible']].sum(axis = 0)

In [ ]:
1 - 275495/ 345077

In [ ]:
new_input_bdf_2017_by_decile

### Méthode 2.3 : En utilisant les revenus calés sur ceux de TaxIPP et les dépenses calées pour 13 Mds€

In [ ]:
from openfisca_france_indirect_taxation.projects.TVA.new_calage_bdf_cn import get_cn_aggregates

In [ ]:
cn_agregates = get_cn_aggregates(2017)

In [ ]:
cn_agregates.loc[cn_agregates.index == 'poste_06']